In [65]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
herodata = pd.read_csv(file_to_load)
df = pd.DataFrame(herodata, columns=['SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'])
df.head()




,SN,Age,Gender,Item ID,Item Name,Price
0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,Ithergue48,24,Male,92,Final Critic,4.88
3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,Iskosia90,23,Male,131,Fury,1.44


In [66]:
totalplayers = herodata['SN'].nunique()
totalplayers

576

In [67]:
aveitemprice = herodata["Price"].mean()
totpurchasevalue = herodata["Price"].sum()
purchasecount = herodata["Price"].count()
itemcount = len(herodata["Item ID"].unique())

summary = pd.DataFrame({"Unique Items": [itemcount],
                                   "Average Price": [aveitemprice],
                                   "Total Purchases": [purchasecount],
                                   "Total Revenue": [totpurchasevalue]})

summary


,Unique Items,Average Price,Total Purchases,Total Revenue
0,183,3.050987,780,2379.77


In [68]:
groupedgenderdf = herodata.groupby(["Gender"])
genderdf = groupedgenderdf.nunique()
total_gender = genderdf["SN"].sum()

count = genderdf["SN"].unique()
percentage = genderdf["SN"]/ total_gender

genderdisplay = pd.DataFrame({"Percentage of Players": percentage,
                            "Count":count})

genderdisplay["Percentage of Players"] = genderdisplay["Percentage of Players"].map("{:,.2%}".format) 

genderdisplay

,Percentage of Players,Count
Gender,,
Female,14.06%,81
Male,84.03%,484
Other / Non-Disclosed,1.91%,11


In [69]:

gender_purchase_total = herodata.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")
gender_average = herodata.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")
gender_counts = herodata.groupby(["Gender"]).count()["Price"].rename("Purchase Count")    
         
summary_table = pd.DataFrame({"Purchase Count": gender_counts,
                             "Average Price": gender_average,
                             "Total Purchase Value": gender_purchase_total,
                             "Average Price Per Person": gender_purchase_total / gender_counts})

summary_table

,Purchase Count,Average Price,Total Purchase Value,Average Price Per Person
Gender,,,,
Female,113,3.203009,361.94,3.203009
Male,652,3.017853,1967.64,3.017853
Other / Non-Disclosed,15,3.346000,50.19,3.346000


In [72]:
agebins = [0,9,14,19,24,29,34,39,100]
agelabels = ["<10","10-14","15-19","20-24","25-29","30-34","35-40","40+"]

dfage=pd.DataFrame(herodata)
dfage = dfage.rename(columns = {'Item ID':'PurchaseID'})




dfage['Age demo']=pd.cut(dfage['Age'],agebins,labels=agelabels)
dfage = dfage.groupby(['Age demo'])



dfage = dfage.PurchaseID.count()

dfage

Age demo
<10       23
10-14     28
15-19    136
20-24    365
25-29    101
30-34     73
35-40     41
40+       13
Name: PurchaseID, dtype: int64

In [78]:

top_spendersvalue = (pd.DataFrame(df.groupby('SN')['Price'].sum())).sort_values("Price",ascending = False)
top_spendersprice= pd.DataFrame(df.groupby('SN')['Price'].mean())
top_spenderscount= pd.DataFrame(df.groupby('SN')['Item ID'].count())

top_spenderdf = pd.merge(top_spendersvalue,top_spendersprice,on = "SN")
top_spenderdf = pd.merge(top_spenderdf,top_spenderscount,on = "SN")

top_spenderdf = top_spenderdf.rename(columns = {'Price_x': 'Total Purchase Value',
                                                'Price_y': 'Average Purchase Price',
                                                'Item ID': 'Purchase Count'})

top_spenderdf['Total Purchase Value'] = top_spenderdf['Total Purchase Value'].map('${:,.2f}'.format)
top_spenderdf['Average Purchase Price'] = top_spenderdf['Average Purchase Price'].map('${:,.2f}'.format)


top_spenderdf = top_spenderdf[["Purchase Count","Average Purchase Price", "Total Purchase Value"]]

top_spenderdf.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [89]:

top_itemprice = pd.DataFrame(df.groupby(["Item ID","Item Name"])['Price'].unique())
top_itemprice['Price'] = top_itemprice['Price'].astype(float)


top_item = pd.DataFrame(df.groupby(["Item ID","Item Name"])['Item ID'].count())
top_item = top_item.rename(columns = {'Item ID': 'Purchase Count'})

top_item = top_item.join(top_itemprice,how = 'outer')
top_item["Tot Purchase Value"] = (top_item["Purchase Count"] * top_item["Price"]).map('${:,.2f}'.format)

top_item["Price"] = top_item["Price"].map('${:,.2f}'.format)
top_item = top_item.sort_values("Purchase Count", ascending = False)

top_item.head()


,,Purchase Count,Price,Tot Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [92]:
top_items = top_item
top_items["Tot Purchase Value"] = (top_items["Tot Purchase Value"].str.replace('$', '')).astype(float)
top_items = top_items.sort_values("Tot Purchase Value", ascending = False)
top_items["Tot Purchase Value"] = top_items["Tot Purchase Value"].map('${:,.2f}'.format)
top_items.head()

,,Purchase Count,Price,Tot Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
